In [1]:
def affine_encrypt(a, b, x, m):
    
    y = (a * x + b) % m
    
    return y

def affine_decrypt(a_inv, b, y, m):
    
    x = a_inv * (y - b) % m
    
    return x

In [2]:
def create_letter_frequency(text):
    n = len(text)
    #freq = vector(ZZ, 26)
    MS = MatrixSpace(ZZ, 26, 2)
    freq = MS()
    
    for i in range(26):
        freq[i,0] = i   # first column of the row stores the alphabet letter itself
    
    for i in range(n):
        index = ord(text[i]) - ord('a')
        freq[index,1] = freq[index,1] + 1    # second column stores the frequency
    
    
    return freq

def bubble_sort(lst):
    n = len(lst.rows())
    ordered_list = lst
    
    #print(str(ordered_list))
    
    for i in range(n):
        for j in [0 .. n-i-2]:
            if ordered_list[j,1] < ordered_list[j + 1,1]:
                temp = ordered_list[j]
                ordered_list[j] = ordered_list[j + 1]
                ordered_list[j + 1] = temp
    #print(str(ordered_list))
    
    return ordered_list

def next_max_idx(arr, curr_max_idx):
    new_max_idx = 0
    temp_min_val = 0
    
    for i in range(len(arr)):
        if arr[i] > temp_min_val and (curr_max_idx == 26 or arr[i] < arr[curr_max_idx]):
            new_max_idx = i
            temp_min_val = arr[new_max_idx]
    
    return new_max_idx

def decrypt_message(y, a_inv, b, m):
    
    x = ""
    
    for i in range(len(y)):
        y_num = ord(y[i]) - ord('a')     # ord('a') = 97
        x_num = affine_decrypt(a_inv, b, y_num, m)
        x     = x + chr(int(x_num) + ord('a'))

    return x
        
def inv(a):
    
    m = 26
    Zm = Integers(m) # Here we give ourselves shorthand for the modular integers
    n = Zm(a) # Here we ask for a as an element of that ring
    n = n^(-1)
    
    print("inverse:", n)
    
    return n

def solve(x1, x2, y1, y2, m):
    
    Zm=Integers(m) # Mod 26
    MS_Zm = MatrixSpace(Zm, 2, 2) # '2, 2' refer to number of rows and columns of the matrices represented by this space
    A_zm = MS_Zm([x1,1,x2,1])  # we solve for a and b in a*x1+b=y1; a*x2+b=y2
    print("a_zm", A_zm)
    
    Y_zm = vector(Zm, [y1, y2])
    print("y_zm", Y_zm)
    
    X_zm = A_zm \ Y_zm
    print("x_zm", X_zm)
    
    return X_zm

In [ ]:
# Below is the frequency-analysis attack on the affine cipher

m = 26
y = "fmxvedkaphferbndkrxrsrefmorudsdkdvshvufedkaprkdlyevlrhhrh"
#y = "LMQETXYEAGTXCTUIEWNCTXLZEWUAISPZYVAPEWLMGQWYAXFTCJMSQCADAGTXLMDXNXSNPJQSYVAPRIQSMHNOCVAXFV".lower()
x = ""
letter_freq = "etaoinshrdlcumwfgypbvkjxqz"
#letter_freq="TH, HE, IN, ER, AN, RE, ED, ON, ES, ST"
freq = create_letter_frequency(y)
#print(freq)
freq = bubble_sort(freq)   # one array will be used to pick the first highest frequency ciphertext letter and the other for the second one

print(freq)
prev_high_freq_ciphertext_lttr = -1


for i in range(m):
    
    high_freq_plaintext_lttr = ord(letter_freq[i]) - ord('a')
    
    print(ord(letter_freq[i])- ord('a'))
    
    for j in range(m): # for j in [i+1 .. m-1]:
        
        if j != i:
            next_high_freq_plaintext_lttr = ord(letter_freq[j]) - ord('a')
            print("plaintext higher freq two letters are: {" + str(letter_freq[i]) + ", " + str(letter_freq[j]) + "}\n")

            high_freq_ciphertext_lttr = 0 # prev_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, prev_high_freq_ciphertext_lttr) # return the idx of the next max in the list of freq. This is the Z_26 representation of the highest freq ciphertext letter
            
            print("prnt ",freq[high_freq_ciphertext_lttr,1])
            while high_freq_ciphertext_lttr < 26 and freq[high_freq_ciphertext_lttr,1] > 0:  # if freq of the letter is > 0

                next_high_freq_ciphertext_lttr = 0 # high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)    # second next max freq ciphertext letter

                while next_high_freq_ciphertext_lttr < 26 and freq[next_high_freq_ciphertext_lttr,1]  > 0:

                    if next_high_freq_ciphertext_lttr != high_freq_ciphertext_lttr:
                        print("ciphertext higher freq two letters are: {" + chr(freq[high_freq_ciphertext_lttr,0] + ord('a')) + ", " + chr(freq[next_high_freq_ciphertext_lttr,0] + ord('a')) + "}\n")
                        
                        
                        print("Vaues" , high_freq_plaintext_lttr, next_high_freq_plaintext_lttr, freq[high_freq_ciphertext_lttr,0], freq[next_high_freq_ciphertext_lttr,0], m)
                        
                        
                        #Matrix Systems of equation LEFT OFF HERE
                        
                        solution = solve(high_freq_plaintext_lttr, next_high_freq_plaintext_lttr, freq[high_freq_ciphertext_lttr,0], freq[next_high_freq_ciphertext_lttr,0], m)

                        if gcd(solution[0], m) == 1:
                            a_inv = inv(solution[0])
                            b = solution[1]
                            x = decrypt_message(y, a_inv, b, m)                    
                            print("a_inv = " + str(a_inv) + ", b = " + str(b) + ", plaintext = " + x)
                            ch = input("Press Enter to continue or E to exit...")
                            if ch == "E" or ch == "e":
                                sys.exit("Exiting the program!!")
                        else:
                            print("a = " + str(solution[0]) + " is not a valid solution\n")

                    next_high_freq_ciphertext_lttr = next_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, next_high_freq_ciphertext_lttr)

                high_freq_ciphertext_lttr = high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)
            

[17  8]
[ 3  7]
[ 4  5]
[ 7  5]
[10  5]
[ 5  4]
[21  4]
[18  3]
[ 0  2]
[11  2]
[12  2]
[15  2]
[20  2]
[23  2]
[ 1  1]
[13  1]
[14  1]
[24  1]
[ 2  0]
[ 6  0]
[ 8  0]
[ 9  0]
[16  0]
[19  0]
[22  0]
[25  0]
4
plaintext higher freq two letters are: {e, t}

prnt  8
ciphertext higher freq two letters are: {r, d}

Vaues 4 19 17 3 26
a_zm [ 4  1]
[19  1]
y_zm (17, 3)
x_zm (6, 19)
a = 6 is not a valid solution

ciphertext higher freq two letters are: {r, e}

Vaues 4 19 17 4 26
a_zm [ 4  1]
[19  1]
y_zm (17, 4)
x_zm (13, 17)
a = 13 is not a valid solution

ciphertext higher freq two letters are: {r, h}

Vaues 4 19 17 7 26
a_zm [ 4  1]
[19  1]
y_zm (17, 7)
x_zm (8, 11)
a = 8 is not a valid solution

ciphertext higher freq two letters are: {r, k}

Vaues 4 19 17 10 26
a_zm [ 4  1]
[19  1]
y_zm (17, 10)
x_zm (3, 5)
inverse: 9
a_inv = 9, b = 5, plaintext = algorithmsarequitegeneraldefinitionsofarithmeticprocesses
